In [47]:
import collections
import math
import os
import random
import zipfile
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import nltk

In [2]:
ppp = nltk.data.load('../Downloads/ppp.txt', encoding='utf8')
words_p = nltk.tokenize.word_tokenize(ppp)[122:]    

In [3]:
alw = nltk.data.load('../Downloads/alw.txt', encoding='utf8')
words_a = nltk.tokenize.word_tokenize(alw)[122:]

In [4]:
words = words_a + words_p

In [5]:
l2 = np.load('./l2_embed3.npy')
centroids = np.load('./centroidsl2.npy').item()
word_embed = np.load('./embed3.npy')
centers = centroids.cluster_centers_
c_labels = centroids.labels_ + len(word_embed)
c_counts = centroids.counts_

In [6]:
embeddings = np.load('./l2embed.npy')

In [7]:
vocabulary_size = 10000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words_a
del words_p # Hint to reduce memory.
print('Most common words (+UNK)', count[:20])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 40], (',', 11834), ('the', 5884), ('.', 5086), ('to', 4942), ('of', 4298), ('and', 4280), ('a', 2623), ('her', 2349), ('in', 2239), ('was', 2186), ('I', 2040), ('she', 1870), ('it', 1737), ('that', 1733), (';', 1732), ('not', 1667), ('you', 1504), ('be', 1414), ('as', 1375)]
Sample data [1316, 11, 3, 3704, 2, 8076, 65, 10, 680, 4] ['CHAPTER', 'I', '.', 'Down', 'the', 'Rabbit-Hole', 'Alice', 'was', 'beginning', 'to']


In [8]:
transition_dict = {}
for index, transition in enumerate(l2):
    f  = transition[-2].astype(int)
    t = transition[-1].astype(int)
    if not (f,t) in transition_dict and c_counts[c_labels[index] - len(word_embed)] > 10:
        transition_dict[tuple([f,t])] = c_labels[index]
        dictionary['l2_' + str(c_labels[index])] = c_labels[index]
        reverse_dictionary[c_labels[index]] = 'l2_' + str(c_labels[index])

reverse_t_dict = dict(zip(transition_dict.values(), transition_dict.keys()))

In [9]:
vocabulary_size += len(centers)

In [37]:
l2_s_vec = []
l2_s_lab = []

sentences_vec = []
sentences_word  = []
sent_word = []
sent_vec = []
indices = []
for index, word in enumerate(filter(lambda word: word in dictionary, words)):
    if word in ['?','.','!']:
        sent_vec += [embeddings[dictionary[word]]]
        sent_word += [word]
        if len(sent_vec) >= 3:
            sn_w = [sent_word[0]]
            sn_v = [sent_vec[0]]
            l2_vec = []
            l2_lab = []
            for index, word in enumerate(sent_word[:-1]):
                f = dictionary[word]
                t = dictionary[sent_word[index+1]]
                pair = (f,t)
                if pair in transition_dict:
                    sn_v += [embeddings[transition_dict[pair]], sent_vec[index+1]]
                    sn_w += ['l2_'+str(transition_dict[pair]), sent_word[index+1]]
                    l2_vec += [embeddings[transition_dict[pair]]]
                    l2_lab += [transition_dict[pair]]
            if len(l2_lab) > 1:
                l2_s_vec += [l2_vec]
                l2_s_lab += [l2_lab]
            sentences_vec += [sn_v]
            sentences_word += [sn_w]
            l2_vec, l2_lab, sn_v, sn_w = [],[],[],[]
        sent_vec = []
        sent_word = []
    elif word in ['Chapter', 'CHAPTER'] and words[index+1] in ['1','I']:
        indices += [len(sentences_vec)]
    else:
        sent_vec += [embeddings[dictionary[word]]]
        sent_word += [word]
#del words

In [38]:
sentences_vec_t = [np.diff(sent, axis=0) for sent in sentences_vec]

words_vec_t = np.array([[[float(dictionary[sent[index]]), 
                         float(dictionary[sent[index+1]])]
                        for index in range(len(sent)-1)] for sent in 
                         sentences_word])


In [39]:
l3_vec_t = [np.diff(sent, axis=0) for sent in l2_s_vec]
l3_lab_t = np.array([[[float(sent[index]), 
                         float(sent[index+1])]
                        for index in range(len(sent)-1)] for sent in 
                         l2_s_lab])

In [30]:
len(l3_vec_t[1])

24

In [41]:
l3 = np.concatenate((l3_vec_t[0], l3_lab_t[0]), axis=1)
for index, sent in enumerate(l3_vec_t[1:]):
    nsent = np.concatenate((sent, l3_lab_t[index+1]), axis=1)
    l3 = np.concatenate((l3, nsent), axis=0)

In [ ]:
LT = np.concatenate((sentences_vec_t[0], words_vec_t[0]), axis=1)
for index, sent in enumerate(sentences_vec_t[1:]):
    nsent = np.concatenate((sent, words_vec_t[index+1]), axis=1)
    l2 = np.concatenate((l2, nsent), axis=0)


In [87]:
mbatch = MiniBatchKMeans(n_clusters=100, batch_size=3000, max_iter=100000)

In [88]:
centroids = mbatch.fit(l3[:,:-2])

/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning: This function is deprecated. Please call randint(0, 78485 + 1) instead
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 78485 + 1) instead
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 78485 + 1) instead
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 78485 + 1) instead
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning: This function is deprecated. Please call randint(0, 78485 + 1) instead
  0,

In [91]:
centroids.counts_.argmax()

8

In [113]:
#centroids = np.load('./centroidsl3.npy').item()

test = pd.DataFrame(l3).groupby(centroids.labels_)
nearests = test.apply(nearest)

In [55]:
def find_nearest_vector(array, value, index):
    indices = np.concatenate((np.arange(index),np.arange(index+1, len(array))))
    sq = np.square(array[indices] - value)
    idx = sq.sum(axis=1).argmin()
    return np.square(array[idx] - value).sum(), idx

In [85]:
def nearest(cluster):
    cluster = cluster.as_matrix()
    if len(cluster) < 2:
        return 
    nn1 = []
    for index, word in enumerate(cluster):
        ret = find_nearest_vector(cluster[:,:-2], cluster[index, :-2], index)
        aa,ab = reverse_t_dict[word[-2]]
        ac,ad = reverse_t_dict[word[-1]]
        ba,bb = reverse_t_dict[cluster[ret[1]][-2]]
        bc,bd = reverse_t_dict[cluster[ret[1]][-1]]
        the_picks = [reverse_dictionary[word] for word in [aa,ab,ac,ad,ba,bb,bc,bd]]
        nn1 += [[ret[0], *the_picks]]
    return pd.DataFrame(nn1, columns=['dist','f1f','f1t','f2f','f2t','t1f','t1t','t2f','t2t'])

In [114]:
no_dup_near = nearests.reset_index().drop_duplicates()

In [115]:
no_dup_near.loc[no_dup_near.dist>0].sort_values(by='dist').head(200)

,level_0,level_1,dist,f1f,f1t,f2f,f2t,t1f,t1t,t2f,t2t
57231,71,591,0.004978,towards,Bingley,Bingley,",",Mr.,Bingley,Bingley,","
22993,19,945,0.007881,I,Office,inspired,think,I,Office,to,think
22384,19,336,0.007881,have,require,to,think,have,require,inspired,think
50534,57,94,0.007881,inspired,think,think,of,to,think,think,of
12506,8,5527,0.008085,;,we,To,must,",",we,To,must
34744,37,333,0.010674,to,his,aunt,",",to,his,aunt,look
52057,61,316,0.011650,‘I,hope,unforgiving,",",I,hope,unforgiving,","
13360,8,6381,0.015838,the,knocking,letter,;,the,knocking,letter,","
60357,75,180,0.016458,strength,here,lovers,that,educated,grave,lovers,that
60527,76,6,0.018422,general,know,indeed,what,you,know,indeed,what


In [116]:
nearests.to_pickle('./nnl2')

## The Plan

### Step 5b
Add a graph on top of the network and begin looking at methods to choose link likelihood

### Step 6
Combine network and word2vec with bigram transitions. Recognize words as transitions. Especially that. Actually. You might even just try recognizing 3 word transitions like $a \xrightarrow{b} c$, that's probably the most concrete place to start. Also play around with a high dropout rate. So train batch. Drop / cluster a bunch then continue.

### Step 7
Try implimenting adding in trigrams inbetween bigrams. Basically, train two contexts once you've trained bigrams.
Also, implement adding of links for likely things. We are trying to optimize the likelihood, given some word, of the links from that word. 

### Step 7b
This is never going to work

### Step 8
Try to forget the hierarchical stuff. Try seeing how high features can get. Assume a fixed vocabulary size, but words not in the vocab can be added as higher level relations. Actually, try to merge transitions with words. Fixed graph size $|V|^2$

### Step 9a
Try to implement some fancy stability constraint stuff

### Step 9b
Try to get this system to recognize authors. You can train a classical neural network, but try it at different levels. Try to recognize authors using only the transitions between words. Even if this just performs decent, what does that mean? What interpretation could one draw. Perhaps this is closer to the idealized style recognition that has been hoped for in the past. Also, we can dry to reduce this to the minimum need transitions or maybe try to find singular identifying transitions per author.

### Step 10
Try to forcast the next sentence from the previous, or the next paragraph from the previous, etc. Try speech generation. Try to untangle mixed up words.